In [1]:
from ultralytics import YOLO

In [3]:
import os
import Augmentor

In [14]:
import os
import Augmentor

# Function to apply augmentation to each subfolder
def augment_images_in_folder(folder_path, num_augmented_images):
    pipeline = Augmentor.Pipeline(folder_path)
    
    # Define augmentation operations
    pipeline.flip_left_right(probability=0.7)
    pipeline.random_brightness(probability=0.5, min_factor=0.5, max_factor=1.4)
    pipeline.rotate90(probability=0.6)
    pipeline.zoom_random(probability=0.5, percentage_area=0.8)
    
    pipeline.sample(num_augmented_images)

# Main script
input_folder = './pill_folder/'

# Iterate through subfolders and apply augmentation
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    
    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        print(f"Augmenting images in folder: {subfolder}")
        augment_images_in_folder(subfolder_path, 100)

print("Augmentation complete.")

Augmenting images in folder: gastric
Initialised with 1 image(s) found.
Output directory set to ./pill_folder/gastric\output.

Processing <PIL.Image.Image image mode=RGB size=2048x1152 at 0x1A58CCEA190>: 100%|█| 100/100 [00:04<00:00, 22.46 Sample


Augmenting images in folder: panadol
Initialised with 1 image(s) found.
Output directory set to ./pill_folder/panadol\output.

Processing <PIL.Image.Image image mode=RGB size=320x240 at 0x1A58B2CD150>: 100%|█| 100/100 [00:00<00:00, 208.86 Samples

Augmentation complete.


In [15]:
import os
import cv2
import yaml

input_root_folder = './pill_folder/'
output_root_folder = './Merged_Images/'
augmented_folder = 'output'  # Adjust this according to the new structure
data_yaml_path = 'data.yaml'

# load the model
yolo = YOLO('models/pill_detection/pill_detection_medium_8batch.pt')

# update the data.yaml file with new classes
def update_data_yaml(data_yaml_path, new_class):

    if not os.path.exists(data_yaml_path):  # Check if the file exists
        with open(data_yaml_path, 'w') as file:  # Create an empty YAML file with the necessary structure
            yaml.dump({'nc': 0, 'names': [], 'train':'', 'test':'', 'val':''}, file)

    with open(data_yaml_path, 'r') as file:  # Load the existing YAML content
        data = yaml.load(file, Loader=yaml.FullLoader)

    if not data['names']:  # Check if the new class already exists (case-insensitive check)
        data['names'] = [new_class]  # Initialize the data['names'] to be a list
        data['nc'] = 1

        if not data['train'] and not data['test'] and not data['val']:
            data['train'] = 'Merged_Images/train/images'
            data['test'] = 'Merged_Images/test/images'
            data['val'] = 'Merged_Images/val/images'
            
        with open(data_yaml_path, 'w') as file:
            yaml.dump(data, file)
        print(f"Class '{new_class}' added to {data_yaml_path}")

    
        
    else:  # Append new classes if data['names'] is already a list
        if new_class not in [existing_class.lower() for existing_class in data['names']]:  # Check for duplicate names
            data['names'].append(new_class)  # Add the new class to the list of names
            data['nc'] = len(set([name.lower() for name in data['names']]))  # Update the count of classes

            with open(data_yaml_path, 'w') as file:  # Save the updated YAML content
                yaml.dump(data, file)
            print(f"Class '{new_class}' added to {data_yaml_path}")

        else:
            print(f"Class '{new_class}' already exists in {data_yaml_path}")

    return data['names']

# Main script
for pill_folder in os.listdir(input_root_folder):  # iterate through the image folders
    pill_folder_path = os.path.join(input_root_folder, pill_folder)
    if os.path.isdir(pill_folder_path):  # Check if the folder is a directory

        if pill_folder != augmented_folder:  # Augmented_Photos folder is for storing the augmented uploaded images
            new_class_name = pill_folder  # Use the name of the image folder as the class name
            class_names = update_data_yaml(data_yaml_path, new_class_name)  # Update YAML file
            one_hot_encoding = {class_name: index for index, class_name in enumerate(class_names)}  # Perform one-hot encoding
        
            augmented_folder_path = os.path.join(pill_folder_path, augmented_folder)  # Path to the augmented images
            
            for pic_name in os.listdir(augmented_folder_path):  # Iterate through files in the augmented subfolder
                
                if pic_name.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if the file is an image
                    img_path = os.path.join(augmented_folder_path, pic_name)  # Adjust the path to the augmented images
                    
                    img = cv2.imread(img_path)  # Load the image
                    original = img.copy()
                    
                    # Check for existing files with the same name
                    existing_count = 0
                    base_name = os.path.splitext(pic_name)[0]  # Remove the extension
                    while True:
                        annotated_txt_path = os.path.join(output_root_folder, pill_folder, f'{base_name}{existing_count}.txt')
                        annotated_img_path = os.path.join(output_root_folder, pill_folder, f'{base_name}{existing_count}.jpeg')

                        if not os.path.exists(os.path.dirname(annotated_txt_path)):
                            os.makedirs(os.path.dirname(annotated_txt_path))

                        if not os.path.exists(os.path.dirname(annotated_img_path)):
                            os.makedirs(os.path.dirname(annotated_img_path))

                        if not os.path.exists(annotated_txt_path) and not os.path.exists(annotated_img_path):
                            break

                        existing_count += 1

                    results = yolo(img)  # Perform pill detection and label

                    if results:
                        for r in results:
                            if len(r.boxes.xywhn.tolist()) > 0:
                                cv2.imwrite(annotated_img_path, original)
                                x, y, w, h = r.boxes.xywhn.tolist()[0]

                                # Save the normalized center x, center y, width, and height value.
                                # This will be used in the training of the model
                                with open(annotated_txt_path, 'w') as file:
                                    line = f"{one_hot_encoding[pill_folder]} {x} {y} {w} {h}\n"
                                    file.write(line)

Class 'gastric' already exists in data.yaml

0: 384x640 1 pill, 2818.1ms
Speed: 11.9ms preprocess, 2818.1ms inference, 26.4ms postprocess per image at shape (1, 3, 384, 640)

0: 640x384 1 pill, 181.9ms
Speed: 4.1ms preprocess, 181.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 pill, 73.8ms
Speed: 3.8ms preprocess, 73.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 pill, 69.9ms
Speed: 3.8ms preprocess, 69.9ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 384)

0: 384x640 1 pill, 71.2ms
Speed: 3.3ms preprocess, 71.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x384 1 pill, 75.0ms
Speed: 6.2ms preprocess, 75.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 384x640 (no detections), 74.5ms
Speed: 1613.4ms preprocess, 74.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 73.6ms
Speed: 2.9ms preprocess, 73.6ms 

In [16]:
import os
import shutil
import pandas as pd

# Set the paths for the original and augmented folders
annotated_folder = "./Merged_Images/"
merged_folder = "./Merged_Images/"

# Create the merged folder if it doesn't exist
if not os.path.exists(merged_folder):
    os.makedirs(merged_folder)

# Function to copy images and their corresponding txt files
def copy_images_and_txt(src_folder, dest_folder):
    for class_folder in os.listdir(src_folder):
        class_path = os.path.join(src_folder, class_folder)
        if os.path.isdir(class_path):
            for root, dirs, files in os.walk(class_path):
                for filename in files:
                    # Check if the file is an image (png, jpg, jpeg, etc.)
                    if any(filename.lower().endswith(ext) for ext in ['.png', '.jpg', '.jpeg']):
                        image_path = os.path.join(root, filename)
                        txt_path = os.path.join(root, filename.rsplit(".", 1)[0] + ".txt")

                        # Copy the image to the merged folder
                        shutil.copy(image_path, dest_folder)

                        # Copy the corresponding txt file if it exists
                        if os.path.exists(txt_path):
                            shutil.copy(txt_path, dest_folder)

# Copy annotated images and txt files
copy_images_and_txt(annotated_folder, merged_folder)

# Create a Pandas DataFrame to store the data
data = []

# Iterate through the merged folder and read txt files
for root, dirs, files in os.walk(merged_folder):
    for filename in files:
        if filename.lower().endswith('.txt'):
            txt_path = os.path.join(root, filename)
            with open(txt_path, 'r') as txt_file:
                content = txt_file.read().split()
                file_name = os.path.splitext(filename)[0]
                class_label = int(content[0])  # Assuming the class is the first value in the txt file
                data.append([file_name, class_label])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Original_File', 'Class'])

In [17]:
df

,Original_File,Class
0,gastric_original_gastric1.jpeg_00eca40d-4b38-4...,0
1,gastric_original_gastric1.jpeg_01885d53-774a-4...,0
2,gastric_original_gastric1.jpeg_05d3f73d-6a37-4...,0
3,gastric_original_gastric1.jpeg_0aed74cb-e021-4...,0
4,gastric_original_gastric1.jpeg_0b10fe46-f549-4...,0
...,...,...
165,panadol_original_panadol.jpeg_f94d1176-646f-4b...,1
166,panadol_original_panadol.jpeg_fec064cf-881a-42...,1
167,panadol_original_panadol.jpeg_ff25b583-0c49-49...,1
168,panadol_original_panadol.jpeg_ff6c522d-d2a4-47...,1


In [18]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Set the paths for the folders to store train, test, and val data
train_folder = "./Merged_Images/train/"
test_folder = "./Merged_Images/test/"
val_folder = "./Merged_Images/val/"

# Create train, test, and val folders if they don't exist
for folder in [train_folder, test_folder, val_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        os.makedirs(os.path.join(folder, "images"))
        os.makedirs(os.path.join(folder, "labels"))

# Use train_test_split with stratify to maintain class distribution
train_df, test_val_df = train_test_split(df, test_size=0.3, stratify=df['Class'], random_state=42)
test_df, val_df = train_test_split(test_val_df, test_size=0.5, stratify=test_val_df['Class'], random_state=42)

def move_files(df, destination_folder):
    for _, row in df.iterrows():
        original_file = row['Original_File']
        image_path = os.path.join(merged_folder, original_file + '.jpeg')
        txt_path = os.path.join(merged_folder, original_file + '.txt')

        # Move images to the destination folder
        shutil.move(image_path, os.path.join(destination_folder, "images"))

        # Move corresponding txt files to the destination folder
        shutil.move(txt_path, os.path.join(destination_folder, "labels"))

# Move images and labels for the train set
move_files(train_df, train_folder)

# Move images and labels for the test set
move_files(test_df, test_folder)

# Move images and labels for the validation set
move_files(val_df, val_folder)

In [19]:
from ultralytics import settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': './', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '8adb090acdbf98e56feb8466776c57f0ade163cc4437393577ad883621649c53', 'sync': True, 'api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [2]:
!yolo task=detect mode=train model=yolov8s.pt data=data.yaml epochs=50 imgsz=640 batch=16

New https://pypi.org/project/ultralytics/8.0.239 available ðŸ˜ƒ Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 ðŸš€ Python-3.11.7 torch-2.1.2 CUDA:0 (NVIDIA GeForce MX330, 2048MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, sav


train: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\train\labels...:   0%|          | 0/119 [00:00<?, ?it/s]
train: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\train\labels... 42 images, 0 backgrounds, 0 corrupt:  35%|###5      | 42/119 [00:00<00:00, 415.01it/s]
train: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\train\labels... 107 images, 0 backgrounds, 0 corrupt:  90%|########9 | 107/119 [00:00<00:00, 547.25it/s]
train: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\train\labels... 119 images, 0 backgrounds, 0 corrupt: 100%|##########| 119/119 [00:00<00:00, 555.94it/s]

val: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\val\labels...:   0%|          | 0/26 [00:00<?, ?it/s]
val: Scanning C:\Users\fangg\OneDrive\Documents\FSP_IVideoAnalytics\Webapp\Merged_Images\val\labels... 26 images, 0 backg

In [ ]:
print("Done")